In [27]:
import os
import time
import matplotlib.pyplot as plt

from collections import OrderedDict
import datetime

In [28]:
def draw_figure(data):
    powers, counts = data["Power (W)"], data["Avg. counts"]
    
    plt.style.use(savelogic.mpl_qudihira_style)

    fig, ax = plt.subplots()

    ax.plot(powers, counts, "o--")
    ax.set_xlabel("Laser power (in W)")
    ax.set_ylabel("Mean counts (kcps)")

    result_linear = fitlogic.make_linear_fit(powers, counts, estimator=fitlogic.estimate_linear)
    ax.plot(powers, result_linear.best_fit, '-', label='Linear fit')

    result_hyper = fitlogic.make_hyperbolicsaturation_fit(powers, counts, 
                                                          estimator=fitlogic.estimate_hyperbolicsaturation)
    ax.plot(powers, result_hyper.best_fit, '-', label='Hyperbolic Saturation fit')

    ax.legend()
    fig.tight_layout()
    return fig, result_linear, result_hyper
    

def save_data(data_to_save, result_linear, result_hyper, fig, tag=None):
    timestamp = datetime.datetime.now()

    parameters = OrderedDict()
    parameters["Linear fit"] = result_linear.fit_report()
    parameters["Hyperbolic fit"] = result_hyper.fit_report()

    data = OrderedDict()
    data["Power (W)"] = data_to_save["Power (W)"]
    data["Avg. counts"] = data_to_save["Avg. counts"]

    filepath = savelogic.get_path_for_module(module_name="Saturation")
    
    if tag:
        filelabel = f"{tag}_saturation"
    else:
        filelabel = "saturation"
    
    print(f"Data saved to {os.path.join(filepath, filelabel)}")

    savelogic.save_data(data, filepath=filepath, parameters=parameters, filelabel=filelabel, 
                        timestamp=timestamp, plotfig=fig, delimiter='\t')

In [50]:
count_freq = counterlogic._count_frequency

current_power = laser_toptica.get_power()

powers = np.linspace(5e-3, 100e-3, 20)
counts = np.zeros(len(powers), dtype=int)

for idx, power in enumerate(powers):
    try: 
        laserlogic.set_power(power)
    except ValueError:
        time.sleep(1)
        laserlogic.set_power(power)
        
    # Wait for laser to settle
    time.sleep(2)

    # Sleep for 5s
    time.sleep(150 / count_freq)

    counts[idx] = np.mean(counterlogic.countdata_smoothed[0, -75:])
    print(f"[{idx+1}/{len(powers)}] Power={power:.3f} W, counts={counts[idx]:.2f}")
    
laserlogic.set_power(current_power)


data = {"Power (W)": powers, "Avg. counts": counts}

[1/20] Power=0.005 W, counts=240.00
[2/20] Power=0.010 W, counts=504.00
[3/20] Power=0.015 W, counts=777.00
[4/20] Power=0.020 W, counts=1334.00
[5/20] Power=0.025 W, counts=1638.00
[6/20] Power=0.030 W, counts=2123.00
[7/20] Power=0.035 W, counts=2476.00
[8/20] Power=0.040 W, counts=2830.00
[9/20] Power=0.045 W, counts=3358.00
[10/20] Power=0.050 W, counts=3678.00
[11/20] Power=0.055 W, counts=4228.00
[12/20] Power=0.060 W, counts=4528.00
[13/20] Power=0.065 W, counts=4930.00
[14/20] Power=0.070 W, counts=5304.00
[15/20] Power=0.075 W, counts=5696.00
[16/20] Power=0.080 W, counts=5954.00
[17/20] Power=0.085 W, counts=6454.00
[18/20] Power=0.090 W, counts=6762.00
[19/20] Power=0.095 W, counts=7229.00
[20/20] Power=0.100 W, counts=7620.00


In [51]:
fig, result_linear, result_hyper = draw_figure(data)
save_data(data, result_linear, result_hyper, fig, tag="background_2")

Data saved to Z:\Data\20220505_FR0612-F2-3B6_uhv\2022\05\20220516\Saturation\background_2_saturation
